In [6]:
import torch
import torch.nn as nn
import altair as alt
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
warnings.filterwarnings("ignore")

In [7]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
torch.cuda.empty_cache()


Using device: cuda


In [19]:
from train import DataSetLoader
from config import get_config
from transformer import Transformer


In [22]:
config = get_config()
train_dataloader, validation_dataloader, test_dataloader, vocab_source, vocab_target = DataSetLoader.get_dataset(config)

Loading raw dataset...
Creating tokenizers...
Looking for tokenizer file: D:\Github\SUMMIT\src\train\opus_books\tokenize\de.json
Found existing tokenizer file, reusing it!
Looking for tokenizer file: D:\Github\SUMMIT\src\train\opus_books\tokenize\en.json
Found existing tokenizer file, reusing it!
Finding longest items...
Longest items found: de: 479, en: 466
Splitting dataset...


In [29]:
model = DataSetLoader.get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)


AttributeError: type object 'DataSetLoader' has no attribute 'get_model'

In [24]:
config = get_config()
file_path = str(Path('.').parent.resolve() / config["TRAIN_DIRECTORY"] / config["datasource"] / config["CHECKPOINT_DIRECTORY"] / config["model_name"])
print(file_path)


D:\Github\SUMMIT\src\train\opus_books\checkpoints\00


In [15]:
#Load pretrained weights
state = torch.load(file_path)

In [ ]:
model.load_state_dict(state['model_state_dict'])